In [2]:
import os
import cv2
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam

# Load the pre-trained emotion model
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128,kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='softmax'))
emotion_model.load_weights('prj.h5')

# Dictionary for emotions
emotion_dict = {0: 'Angry', 1: 'Sad', 2: 'Surprise', 3: 'Happy', 4: 'Neutral', 5: 'Disgusted', 6: 'Fear'}

# Load emojis from the current working directory
cur_path = os.getcwd()  # This gets the current working directory
emoji_dist = {
    0: os.path.join(cur_path, "emojis", "angry.png"),
    1: os.path.join(cur_path, "emojis", "sad.png"),
    2: os.path.join(cur_path, "emojis", "surprise.png"),
    3: os.path.join(cur_path, "emojis", "happy.png"),
    4: os.path.join(cur_path, "emojis", "neutral.png"),
    5: os.path.join(cur_path, "emojis", "disgust.png"),
    6: os.path.join(cur_path, "emojis", "fear.png")
}

# Initialize video capture from the webcam
cap = cv2.VideoCapture(1)

# Load the full body cascade classifier
fullbody_cascade = cv2.CascadeClassifier("C:/Users/Divyansh/OneDrive/Desktop/face/haarcascade_fullbody.xml")

while True:
    ret, frame = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect full bodies in the frame
    bodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in bodies:
        # Crop and resize the detected full body to 48x48 pixels for emotion prediction
        body = gray[y:y+h, x:x+w]
        body = cv2.resize(body, (48, 48))
        body = body.reshape(1, 48, 48, 1) / 255.0

        # Predict the emotion for the detected full body
        prediction = emotion_model.predict(body)
        emotion_label = np.argmax(prediction)
        emotion_emoji_path = emoji_dist[emotion_label]
        emoji_img = cv2.imread(emotion_emoji_path)

        # Resize and overlay the emoji on the detected body
        emoji_img = cv2.resize(emoji_img, (w, h))
        frame[y:y+h, x:x+w] = emoji_img

    # Display the video with the emotion-emoji overlay
    cv2.imshow('Emotion-Emoji', frame)

    # Exit the loop if the 'q' key is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release video capture and close windows
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [3]:
pip install tensorflow


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cv2
import numpy as np
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ignore warnings
warnings.filterwarnings("ignore")

# Load the emotion model
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('prj.h5')  # Ensure this path is correct

# Emotion dictionary
emotion_dict = {0: 'Angry', 1: 'Sad', 2: 'Surprise', 3: 'Happy', 4: 'Neutral', 5: 'Disgusted', 6: 'Fear'}

# Set emoji paths
cur_path = os.getcwd()  # Get current working directory
emoji_dist = {
    0: os.path.join(cur_path, "emojis", "angry.png"),
    1: os.path.join(cur_path, "emojis", "sad.png"),
    2: os.path.join(cur_path, "emojis", "surprise.png"),
    3: os.path.join(cur_path, "emojis", "happy.png"),
    4: os.path.join(cur_path, "emojis", "neutral.png"),
    5: os.path.join(cur_path, "emojis", "disgust.png"),
    6: os.path.join(cur_path, "emojis", "fear.png")
}

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)  # Change to 1 if you want to test the second camera

# Load the frontal-face cascade classifier
face_cascade = cv2.CascadeClassifier("C:/Users/Divyansh/OneDrive/Desktop/face/haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # For each detected face, focus on the face area
        face = gray[y:y+h, x:x+w]
        
        # Check if the face region is not empty
        if face.size == 0:
            continue
        
        # Resize and prepare the face for prediction
        face = cv2.resize(face, (48, 48))
        face = face.reshape(1, 48, 48, 1) / 255.0
        
        # Predict emotion
        prediction = emotion_model.predict(face)
        emotion_label = np.argmax(prediction)
        emotion_emoji_path = emoji_dist[emotion_label]
        emoji_img = cv2.imread(emotion_emoji_path)

        # Check if the emoji image was loaded successfully
        if emoji_img is None:
            print(f"Error loading emoji image: {emotion_emoji_path}")
            continue  # Skip this iteration if emoji loading fails

        # Resize the emoji to fit the detected face region
        emoji_img = cv2.resize(emoji_img, (w, h))
        frame[y:y+h, x:x+w] = emoji_img  # Replace face area with the emoji
        
    # Display the frame with emojis
    cv2.imshow('Emotion-Emoji', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

KeyboardInterrupt: 

In [4]:
emoji_dist = {
    0: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\angry.png",
    1: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\sad.png",
    2: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\surprise.png",
    3: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\happy.png",
    4: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\neutral.png",
    5: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\disgust.png",
    6: r"C:\Users\Divyansh\OneDrive\Desktop\face\emojis\fear.png"
}


In [5]:
print(f"Loading emoji image: {emoji_dist[emotion_label]}")
emoji_img = cv2.imread(emoji_dist[emotion_label])


Loading emoji image: C:\Users\Divyansh\OneDrive\Desktop\face\emojis\fear.png


In [1]:
import os
import cv2
import numpy as np
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Ignore warnings
warnings.filterwarnings("ignore")

# Load the emotion model
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('prj.h5')  # Ensure this path is correct

# Emotion dictionary
emotion_dict = {0: 'Angry', 1: 'Sad', 2: 'Surprise', 3: 'Happy', 4: 'Neutral', 5: 'Disgusted', 6: 'Fear'}

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)  # Change to 1 if you want to test the second camera

# Load the frontal-face cascade classifier
face_cascade = cv2.CascadeClassifier("C:/Users/Divyansh/OneDrive/Desktop/important/face/haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Convert to grayscale only once per frame
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Focus on the face area for emotion detection
        face = gray[y:y+h, x:x+w]
        
        # Check if the face region is valid
        if face.size == 0:
            continue
        
        # Resize, normalize, and reshape the face for prediction
        face = cv2.resize(face, (48, 48))
        face = face.reshape(1, 48, 48, 1) / 255.0
        
        # Predict emotion
        prediction = emotion_model.predict(face)
        emotion_label = np.argmax(prediction)
        emotion_text = emotion_dict[emotion_label]
        confidence = np.max(prediction)  # Get the confidence of the prediction
        
        # Display emotion text and confidence on the frame
        label = f"{emotion_text} ({confidence*100:.2f}%)"
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
    # Display the frame with emotion text
    cv2.imshow('Emotion Detection', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━

In [1]:
import os

import cv2


import numpy as np

import warnings
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ignore warnings
warnings.filterwarnings("ignore")

# Load the emotion model
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))
emotion_model.load_weights('prj.h5')  # Ensure this path is correct

# Emotion dictionary
emotion_dict = {0: 'Angry', 1: 'Sad', 2: 'Surprise', 3: 'Happy', 4: 'Neutral', 5: 'Disgusted', 6: 'Fear'}

# Set emoji paths
cur_path = os.getcwd()  # Get current working directory
emoji_dist = {
    0: os.path.join(cur_path, "emojis", "angry.png"),
    1: os.path.join(cur_path, "emojis", "sad.png"),
    2: os.path.join(cur_path, "emojis", "surprise.png"),
    3: os.path.join(cur_path, "emojis", "happy.png"),
    4: os.path.join(cur_path, "emojis", "neutral.png"),
    5: os.path.join(cur_path, "emojis", "disgust.png"),
    6: os.path.join(cur_path, "emojis", "fear.png")
}

# Initialize video capture from the webcam
cap = cv2.VideoCapture(0)  # Change to 1 if you want to test the second camera

# Load the frontal-face cascade classifier
face_cascade = cv2.CascadeClassifier("C:/Users/Divyansh/OneDrive/Desktop/face/haarcascade_frontalface_default.xml")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # For each detected face, focus on the face area
        face = gray[y:y+h, x:x+w]
        
        # Check if the face region is not empty
        if face.size == 0:
            continue
        
        # Resize and prepare the face for prediction
        face = cv2.resize(face, (48, 48))
        face = face.reshape(1, 48, 48, 1) / 255.0
        
        # Predict emotion
        prediction = emotion_model.predict(face)
        emotion_label = np.argmax(prediction)
        emotion_emoji_path = emoji_dist[emotion_label]
        emoji_img = cv2.imread(emotion_emoji_path)

        # Resize the emoji to fit the detected face region
        emoji_img = cv2.resize(emoji_img, (w, h))
        frame[y:y+h, x:x+w] = emoji_img  # Replace face area with the emoji
        
    # Display the frame with emojis
    cv2.imshow('Emotion-Emoji', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture and destroy all windows
cap.release()
cv2.destroyAllWindows()

: 